Nenwin-project (NEural Networks WIthout Neurons) for
the AI Honors Academy track 2020-2021 at the TU Eindhoven.

Author: Lulof Pirée
March 2021

Copyright (C) 2020 Lulof Pirée, Teun Schilperoort

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU Affero General Public License as published
by the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU Affero General Public License for more details.

You should have received a copy of the GNU Affero General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

# Backprob experiment

This file provides a full run of backpropagating though an entire `NenwinModel`.

Author: Lulof Pirée

In [2]:
import torch
import torch.nn as nn

from nenwin.all_particles import Marble, Node, MarbleEaterNode, MarbleEmitterNode
from nenwin.model import NenwinModel
from nenwin.attraction_functions.attraction_functions import NewtonianGravity

## Approach

1. Decide how to capture output and network architecture.
1. Set up the model and the particles.
1. Add a loss function.
1. Visualize the model.
1. Run the model.
1. Backpropagate and update.
1. Compare differences in model.

### Design
Let's create a ring of Nodes, say 5 of them, and put one Marble in there. Not in the center, just somewhere within the circle of space (let's keep it 2D) enclosed by the ring of Nodes.
Now let the loss simply be the distance of the Marble to the center of the circle **after 5 seconds from the start**.

One likely result -- if everything works -- is that the Marble simply is placed stationary on this position. But that would already be some successfull optimization!

In [11]:
zero = torch.tensor([0, 0], dtype=torch.float)
mass = 10

center = torch.tensor([0, 0], dtype=torch.float, requires_grad=False)
node_positions = [torch.tensor(pos, dtype=torch.float) for pos in ((0, 10), (10, 0), (0, -10), (-10, 0))]
marble_pos = torch.tensor([2, 2], dtype=torch.float)

nodes = [Node(pos, zero, zero, mass, NewtonianGravity(), 1, 1, 1, 1) 
    for pos in node_positions]

marble = Marble(marble_pos, zero, zero, mass, NewtonianGravity(), None)

model = NenwinModel(nodes, (marble,))

optimizer = torch.optim.Adam(model.parameters())

torch.autograd.set_detect_anomaly(True)

step_size = 0.5

for epoch in range(25):
    optimizer.zero_grad()
    model.reset()
    
    t = 0
    while t <= 5:
        model.make_timestep(step_size)
        t += step_size
        print(t)

    loss = torch.mean(torch.pow(marble.pos - center, 2))
    print(loss)
    loss.backward()
    optimizer.step()
    print(loss)


0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5
5.0
5.5
tensor(33.8699, grad_fn=<MeanBackward0>)
tensor(33.8699, grad_fn=<MeanBackward0>)
0.5
1.0
1.5
2.0
2.5
3.0
3.5
4.0
4.5
5.0
5.5
tensor(38.8427, grad_fn=<MeanBackward0>)


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor []] is at version 1; expected version 0 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!